In [31]:
import datasets

dataset = datasets.load_dataset("philschmid/markdown-documentation-transformers")

dataset

df = dataset["train"].to_pandas()

df.head()

,title,url,markdown,html,crawlDate
0,🤗 Transformers,https://huggingface.co/docs/transformers/v4.34...,# 🤗 Transformers\n\nState-of-the-art Machine L...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:32.994Z
1,Quick tour,https://huggingface.co/docs/transformers/v4.34...,# Quick tour\n\nGet up and running with 🤗 Tran...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:34.381Z
2,Installation,https://huggingface.co/docs/transformers/v4.34...,# Installation\n\nInstall 🤗 Transformers for w...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:34.498Z
3,Pipelines for inference,https://huggingface.co/docs/transformers/v4.34...,# Pipelines for inference\n\nThe [pipeline()](...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:35.128Z
4,Load pretrained instances with an AutoClass,https://huggingface.co/docs/transformers/v4.34...,# Load pretrained instances with an AutoClass\...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:35.231Z


In [32]:
df = df.rename(columns={'html': 'context'})
df['context_length'] = df['context'].apply(lambda x: len(x))
# Documents greater than 300k give us a 500 error
df = df[df['context_length'] <= 300_000]
# All 300 documents takes about an 1.5 hours to ingest
# So let's sample 50 for now
df = df.sample(n=50, random_state=42).reset_index(drop=True)
df.to_parquet("data/transformers.parquet")
df.head()

,title,url,markdown,context,crawlDate,context_length
0,,https://huggingface.co/docs/transformers/v4.34...,The documentation page MODEL\_DOC/SEQUENCE\_CL...,<html><head></head><body>The documentation pag...,2023-10-05T13:33:56.036Z,299
1,Translation,https://huggingface.co/docs/transformers/v4.34...,# Translation\n\nTranslation converts a sequen...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:33:53.214Z,227196
2,Run training on Amazon SageMaker,https://huggingface.co/docs/transformers/v4.34...,# Run training on Amazon SageMaker\n\n![Huggin...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:41.558Z,145189
3,Create a custom architecture,https://huggingface.co/docs/transformers/v4.34...,# Create a custom architecture\n\nAn [`AutoCla...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:40.917Z,216090
4,Image Processor,https://huggingface.co/docs/transformers/v4.34...,# Image Processor\n\nAn image processor is in ...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:31:11.122Z,239715


In [33]:
df.describe()

,context_length
count,50.00000
mean,178385.28000
std,92226.00916
min,253.00000
25%,159669.25000
50%,209115.50000
75%,240418.50000
max,296460.00000


In [34]:
from counter import get_and_increment_counter
from llama_stack_client import LlamaStackClient
from llama_stack_client.types.memory_insert_params import Document

client = LlamaStackClient(
    base_url="http://localhost:5001",
)

providers = client.providers.list()
memory_banks_response = client.memory_banks.list()

bank_id = f"bank_{get_and_increment_counter()}"
provider = providers["memory"][0]
client.memory_banks.register(
    memory_bank_id=bank_id,
    params={
        "embedding_model": "all-MiniLM-L6-v2",
        # Is the default for agent config: https://github.com/meta-llama/llama-stack/blob/66d8f4ffd126bff668434b314892a99fe854a034/llama_stack/providers/inline/agents/meta_reference/agent_instance.py#L668
        "chunk_size_in_tokens": 512,
    },
    provider_id=provider.provider_id,
)
bank_id

'bank_59'

In [35]:
documents = [
    Document(
        document_id=str(idx),
        content=context,
        mime_type="text/plain",
        metadata={},
    )
    for idx, context in zip(df.index, df["context"])
]
documents[:5]

[{'document_id': '0',
  'content': '<html><head></head><body>The documentation page MODEL_DOC/SEQUENCE_CLASSIFICATION.MD doesn’t exist in v4.34.0, but exists on the main version. Click <a href="/docs/transformers/main/en/model_doc/sequence_classification.md">here</a> to redirect to the main version of the documentation.</body></html>',
  'mime_type': 'text/plain',
  'metadata': {}},
 {'document_id': '1',
  'content': '<!DOCTYPE html><html class=""><head>\n\t\t<meta charset="utf-8">\n\t\t<meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no">\n\t\t<meta name="description" content="We’re on a journey to advance and democratize artificial intelligence through open source and open science.">\n\t\t<meta property="fb:app_id" content="1321688464574422">\n\t\t<meta name="twitter:card" content="summary_large_image">\n\t\t<meta name="twitter:site" content="@huggingface">\n\t\t<meta property="og:title" content="Translation">\n\t\t<meta property="og:type" content="

In [36]:
from tqdm import tqdm

for i in tqdm(range(len(documents))):
    client.memory.insert(
        bank_id=bank_id,
        documents=[documents[i]],
    )

  0%|                                                                                    | 0/50 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████| 50/50 [00:47<00:00,  1.06it/s]
